In [1]:
import pandas as pd
import numpy as np

gff3Cols=["seqid","source","type","start","end","score","strand","phase","attributes"]

In [2]:
# read fasta index
chrDict = dict()
with open("/home/sparrow/genomicData/hg38/hg38_p8.fa.fai") as inFP:
    for line in inFP.readlines():
        lineCols = line.split("\t")
        chrDict.setdefault(lineCols[0],0)
        chrDict[lineCols[0]]=int(lineCols[1])
        
chrDict["chr1"]

248956422

In [3]:
df=pd.read_csv("./hg38_p8.biotype_flt.cls.gff3",sep="\t",skiprows=7,names=gff3Cols)

df["parent"]=df.attributes.str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
df=df[df["parent"].isnull()].reset_index(drop=True)
df.drop_duplicates(["seqid","start","end","strand"],inplace=True)
df["id"]=df.attributes.str.split("ID=",expand=True)[1].str.split(";",expand=True)[0]
df.drop_duplicates("id",inplace=True)
df.reset_index(drop=True,inplace=True)
df.drop(["id","parent"],axis=1)
df.head()

,seqid,source,type,start,end,score,strand,phase,attributes,parent,id
0,chr1,gffcl,locus,34611,36081,.,-,.,ID=RLOC_00000002;locus=RLOC_00000002;genes=FAM...,None,RLOC_00000002
1,chr1,gffcl,locus,51943,53959,.,+,.,ID=RLOC_00000003;locus=RLOC_00000003;genes=LOC...,None,RLOC_00000003
2,chr1,gffcl,locus,69091,70008,.,+,.,ID=RLOC_00000004;locus=RLOC_00000004;genes=OR4...,None,RLOC_00000004
3,chr1,gffcl,locus,134773,140566,.,-,.,ID=RLOC_00000005;locus=RLOC_00000005;genes=LOC...,None,RLOC_00000005
4,chr1,gffcl,locus,181049,184258,.,+,.,ID=RLOC_00000006;locus=RLOC_00000006;genes=LOC...,None,RLOC_00000006


In [4]:
df["start"]=df.start.astype(int)
df["end"]=df.end.astype(int)
# add start and end for each strand
for chrID in set(df["seqid"]):
    df=df.append({"seqid":chrID,"source":"new","type":"locus","start":0,"end":1,"score":".","strand":"+","phase":".","attributes":"none=0"}, ignore_index=True)
    df=df.append({"seqid":chrID,"source":"new","type":"locus","start":0,"end":1,"score":".","strand":"-","phase":".","attributes":"none=0"}, ignore_index=True)
    df=df.append({"seqid":chrID,"source":"new","type":"locus","start":chrDict[chrID]-1,"end":chrDict[chrID]-2,"score":".","strand":"+","phase":".","attributes":"none=0"}, ignore_index=True)
    df=df.append({"seqid":chrID,"source":"new","type":"locus","start":chrDict[chrID]-1,"end":chrDict[chrID]-2,"score":".","strand":"-","phase":".","attributes":"none=0"}, ignore_index=True)


df.drop(["parent","id"],axis=1,inplace=True)
df.tail()

,seqid,source,type,start,end,score,strand,phase,attributes
111513,chr8_KI270811v1_alt,new,locus,292435,292434,.,-,.,none=0
111514,chr12_KI270834v1_alt,new,locus,0,1,.,+,.,none=0
111515,chr12_KI270834v1_alt,new,locus,0,1,.,-,.,none=0
111516,chr12_KI270834v1_alt,new,locus,119497,119496,.,+,.,none=0
111517,chr12_KI270834v1_alt,new,locus,119497,119496,.,-,.,none=0


In [5]:
df.sort_values(by=["seqid","strand","start","end"],ascending=True,inplace=True)
df.head()

,seqid,source,type,start,end,score,strand,phase,attributes
111450,chr1,new,locus,0,1,.,+,.,none=0
1,chr1,gffcl,locus,51943,53959,.,+,.,ID=RLOC_00000003;locus=RLOC_00000003;genes=LOC...
2,chr1,gffcl,locus,69091,70008,.,+,.,ID=RLOC_00000004;locus=RLOC_00000004;genes=OR4...
4,chr1,gffcl,locus,181049,184258,.,+,.,ID=RLOC_00000006;locus=RLOC_00000006;genes=LOC...
5,chr1,Gnomon,mRNA,181651,184258,.,+,.,ID=rna38;geneID=gene16;gene_name=LOC102725121;...


In [6]:
df["next_start"]=df["start"].shift(-1)
df["next_seqid"]=df["seqid"].shift(-1)
df["next_strand"]=df["strand"].shift(-1)
df["next_end"] = df["end"].shift(-1)
df=df.dropna(axis=0)
df["next_start"]=df.next_start.astype(int)
df["next_end"]=df.next_end.astype(int)
df.head()

,seqid,source,type,start,end,score,strand,phase,attributes,next_start,next_seqid,next_strand,next_end
111450,chr1,new,locus,0,1,.,+,.,none=0,51943,chr1,+,53959
1,chr1,gffcl,locus,51943,53959,.,+,.,ID=RLOC_00000003;locus=RLOC_00000003;genes=LOC...,69091,chr1,+,70008
2,chr1,gffcl,locus,69091,70008,.,+,.,ID=RLOC_00000004;locus=RLOC_00000004;genes=OR4...,181049,chr1,+,184258
4,chr1,gffcl,locus,181049,184258,.,+,.,ID=RLOC_00000006;locus=RLOC_00000006;genes=LOC...,181651,chr1,+,184258
5,chr1,Gnomon,mRNA,181651,184258,.,+,.,ID=rna38;geneID=gene16;gene_name=LOC102725121;...,182032,chr1,+,184258


In [7]:
before = 1
after  = 0

while(before>after):
    df.sort_values(by=["seqid","strand","start","end"],ascending=True,inplace=True)
    
    df["next_start"]=df["start"].shift(-1)
    df["next_seqid"]=df["seqid"].shift(-1)
    df["next_strand"]=df["strand"].shift(-1)
    df["next_end"] = df["end"].shift(-1)
    df=df.dropna(axis=0)
    df["next_start"]=df.next_start.astype(int)
    df["next_end"]=df.next_end.astype(int)
    
    df.reset_index(drop=True,inplace=True)
    df["prev_start"]=df["start"].shift(1)
    df["prev_end"]=df["end"].shift(1)
    df["prev_seqid"]=df["seqid"].shift(1)
    df["prev_strand"]=df["strand"].shift(1)

    # now need to extend them as far as possible
    df["end"]=np.where((df["seqid"]==df["next_seqid"]) & \
                        (df["strand"]==df["next_strand"]) & \
                        (df["end"]>df["next_start"]),df["next_end"],df["end"])
    df["prev_start"]=np.where(df["prev_start"].isnull(),df["start"],df["prev_start"])
    df["prev_end"]=np.where(df["prev_end"].isnull(),df["end"],df["prev_end"])
    df["prev_seqid"]=np.where(df["prev_seqid"].isnull(),df["seqid"],df["prev_seqid"])
    df["prev_strand"]=np.where(df["prev_strand"].isnull(),df["strand"],df["prev_strand"])
    df["prev_start"]=df["prev_start"].astype(int)
    df["prev_end"]=df["prev_end"].astype(int)
    before=len(df)
    df_first = df.iloc[0]
    df=df[~((df["seqid"]==df["prev_seqid"]) & \
              (df["strand"]==df["prev_strand"]) & \
              (df["start"]>=df["prev_start"]) & \
              (df["end"]<=df["prev_end"]))]
    df=df.append(df_first)
    after=len(df)

# finally create new introns and write everything to the new locus gff for masking
df.sort_values(by=["seqid","strand","start","end"],ascending=True,inplace=True)
df.reset_index(drop=True,inplace=True)
df["next_start"]=df["start"].shift(-1)
df["next_seqid"]=df["seqid"].shift(-1)
df["next_strand"]=df["strand"].shift(-1)
df["start"]=df["end"]
df["end"]=np.where((df["seqid"]==df["next_seqid"]) & \
                        (df["strand"]==df["next_strand"]),df["next_start"],df["end"])
df

,seqid,source,type,start,end,score,strand,phase,attributes,next_start,next_seqid,next_strand,next_end,prev_start,prev_end,prev_seqid,prev_strand
0,chr1,new,locus,1,51943.0,.,+,.,none=0,51943.0,chr1,+,53959,0,1,chr1,+
1,chr1,gffcl,locus,53959,69091.0,.,+,.,ID=RLOC_00000003;locus=RLOC_00000003;genes=LOC...,69091.0,chr1,+,70008,0,1,chr1,+
2,chr1,gffcl,locus,70008,181049.0,.,+,.,ID=RLOC_00000004;locus=RLOC_00000004;genes=OR4...,181049.0,chr1,+,184258,51943,53959,chr1,+
3,chr1,gffcl,locus,184258,200442.0,.,+,.,ID=RLOC_00000006;locus=RLOC_00000006;genes=LOC...,200442.0,chr1,+,203983,69091,70008,chr1,+
4,chr1,gffcl,locus,203983,778760.0,.,+,.,ID=RLOC_00000007;locus=RLOC_00000007;genes=LOC...,778760.0,chr1,+,810065,181049,184258,chr1,+
5,chr1,gffcl,locus,810065,817371.0,.,+,.,ID=RLOC_00000017;locus=RLOC_00000017;genes=LOC...,817371.0,chr1,+,819834,200442,203983,chr1,+
6,chr1,gffcl,locus,819834,827591.0,.,+,.,ID=RLOC_00000018;locus=RLOC_00000018;genes=FAM...,827591.0,chr1,+,859446,778760,810065,chr1,+
7,chr1,BestRefSeq,ncRNA,859446,860226.0,.,+,.,ID=rna87;geneID=gene50;gene_name=LINC01128;Dbx...,860226.0,chr1,+,868202,817371,819834,chr1,+
8,chr1,Gnomon,ncRNA,868202,905115.0,.,+,.,ID=rna89;geneID=gene51;gene_name=LOC107984850;...,905115.0,chr1,+,920466,827591,859446,chr1,+
9,chr1,Gnomon,ncRNA,920466,925741.0,.,+,.,ID=rna93;geneID=gene54;gene_name=LOC284600;Dbx...,925741.0,chr1,+,944581,860226,868202,chr1,+


In [8]:
last_chr = df.iloc[len(df)-1].seqid
print(last_chr)
df.set_value(len(df)-1, 'end', chrDict[last_chr]-1)
df.tail()

chrY


/home/sparrow/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


,seqid,source,type,start,end,score,strand,phase,attributes,next_start,next_seqid,next_strand,next_end,prev_start,prev_end,prev_seqid,prev_strand
37328,chrY,gffcl,locus,13986512,14793642.0,.,-,.,ID=RLOC_00033948;locus=RLOC_00033948;genes=VCY...,14793642.0,chrY,-,14804033,13230770,13480670,chrY,-
37329,chrY,gffcl,locus,14804033,15247106.0,.,-,.,ID=RLOC_00033951;locus=RLOC_00033951;genes=NLG...,15247106.0,chrY,-,15365135,13985772,13986512,chrY,-
37330,chrY,gffcl,locus,15365135,17554144.0,.,-,.,ID=RLOC_00033952;locus=RLOC_00033952;genes=LOC...,17554144.0,chrY,-,17558838,14793642,14804033,chrY,-
37331,chrY,gffcl,locus,17558838,17575161.0,.,-,.,ID=RLOC_00033954;locus=RLOC_00033954;genes=LOC...,17575161.0,chrY,-,17579452,15247106,15365135,chrY,-
37332,chrY,gffcl,locus,17579452,57227414.0,.,-,.,ID=RLOC_00033955;locus=RLOC_00033955;genes=FAM...,NaN,NaN,NaN,17689263,17554144,17558838,chrY,-


In [9]:
df["start"]=df.start.astype(int)
df["end"]=df.end.astype(int)
df.head(6)
df=df[~(df["start"]==df["end"])].reset_index(drop=True)

In [11]:
df.iloc[1603]

seqid               chr1
source               new
type               locus
start                  1
end                34611
score                  .
strand                 -
phase                  .
attributes        none=0
next_start         34611
next_seqid          chr1
next_strand            -
next_end           36081
prev_start     248956421
prev_end       248956420
prev_seqid          chr1
prev_strand            +
Name: 1603, dtype: object

In [12]:
df[gff3Cols].to_csv("./hg38_p8.biotype_flt.cls.loci.gff3",sep='\t',index=False,header=False)